<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Connectivity Workshop SWDB 2017 </h1> 
<h3 align="center">Monday, August 21, 2017</h3>


<p>This notebook will introduce you to the Allen Mouse Brain Connectivity Atlas and its associated SDK.  As an example use case we will consider the question "Where do primary visual cortical neurons project?".

In [4]:
# please make sure your drive_path is set, so that the notebook can find the data files on the hard drive

# OS X
# drive_path = '/Volumes/Brain2017/data/dynamic-brain-workshop/mouse_connectivity_cache'

# Windows (a good guess)
# drive_path = 'e:/data/dynamic-brain-workshop/mouse_connectivity_cache'

# Linux (will vary; the following is possibly what Ubuntu will do)
# drive_path = '/media/Brain2017/data/dynamic-brain-workshop/mouse_connectivity_cache'

# AWS
drive_path = '/data/dynamic-brain-workshop/mouse_connectivity_cache/'

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Mouse Connectivity SDK</h2>

<p>
Connectivity data can be accessed <a href="http://connectivity.brain-map.org/">on the web</a>. This is useful for browsing and familiarizing yourself with the data, but for more in-depth analyses, it is often better to access data programmatically. The <a href="https://alleninstitute.github.io/AllenSDK/">AllenSDK</a> provides Python-based tools to support that approach, and this notebook will introduce you to them.<p>

<p>
If you have not installed the `allensdk` python module yet, do so from a command prompt:</p>

<p>
   `> pip install allensdk` </p>
    
</div>

In [ ]:
# imports!

# pandas for manipulating / viewing tables
import pandas as pd

# numpy, which needs no introduction
import numpy as np

# os for dealing with the file system
import os

# easy plotting
import matplotlib.pyplot as plt

# MouseConnectivityCache for downloading and storing mouse connectivity data
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

# OntologiesApi
from allensdk.api.queries.ontologies_api import OntologiesApi

In [ ]:
# settings!

%matplotlib inline

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<center><h3>Where is the primary visual area in the mouse?</h3></center>

We'll answer this question in two ways:
<ul>
<li>relatively: We want to know which structure(s) contain the primary visual.</li>
<li>absolutely: We want to know where the primary visual area is located in physical space.</li>
</ul>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; "><h4>The caching pattern:</h4>
<p>We will be using the MouseConnectivityCache class to download and store data. This class requires from us:

<ul>
<li>a manifest file path. This path determines where data will be stored. Once you have instantiated your cache once, you can access downloaded data again by instantiating a new cache with the same manifest path.  We will be using pre-populated data directories.</li>
<ul>
<li><i>A MouseConnectivityCache is used to copy and cache data from the cell types database. If you create the MouseConnectivityCache instance with no arguments, then data is downloaded from the online repository:
mcc = MouseConnectivityCache()</i></li>
<p><li><i>Because we have limited internet bandwidth, the data you need is already provided on your external hard drive and AWS. To ensure this local data is used instead of the online repository, create the MouseConnectivityCache instance using the manifest_file argument:</i></li></p></ul>

<li>a resolution for the data volumes that we will be downloading. The possible (isometric) resolutions are 10, 25, 50, and 100-microns, with 25 the default.</li>
</ul>

<p>As usual, you can use the built in help function to see the complete documentation.</p></div>

In [ ]:
# specify your path and isometric resolution
manifest_file = os.path.join(drive_path, 'mouse_connectivity_manifest.json')
resolution = 25

# instantiate the cache object
# after this step, the manifest file you specified should exist on your filesystem
mcc = MouseConnectivityCache(manifest_file=manifest_file, resolution=resolution)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; "><h3><center>The structure tree</center></h3>
Anatomists at the Allen Institute have divided the adult mouse brain into ~1300 distinct structures. These are arranged in a tree, where coarsely specified structures (e.g. 'Isocortex') are near the root and finely specified structures (e.g "Primary visual area, layer 4") make up the leaves.

<a href="http://atlas.brain-map.org/atlas?atlas=1">Here</a> is a handy visualization of the tree (and atlas!).

<p>You can download and access the structure tree by calling get_structure_tree on your MouseConnectivityCache instance.</div>

In [ ]:
# get the structure tree
# this step should produce a file called 'structures.json' in the same directory as your manifest file
structure_tree = mcc.get_structure_tree()

In [ ]:
# view the structure record for primary visual cortex
visp = structure_tree.get_structures_by_name(['Primary visual area'])

# the data structure returned by this method is a list of dictionaries:
visp

In [ ]:
# we can make a nicer table view using pandas:
pd.DataFrame(visp)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
See <a href='http://alleninstitute.github.io/AllenSDK/_static/examples/nb/reference_space.html'>here</a> for definitions of these fields. The most relevant ones for us right now are:

<ul>
<li>id: A unique integer identifier for this structure</li>
<li>name: The structure's full name</li>
<li>acronym: A shortened version of the structure's name</li>
<li>structure_id_path: a list of structures from the root of the tree (left) to the selected structure record.</li>
</ul>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h4>Mapping ids to names</h4>

<p>We can identify a structure by:
<ul>
<li>Its id</li>
<li>Its name</li>
<li>Its acronym</li>
</ul>

<p>Frequently, we want to switch between these. For instance, we might want to know the name of VISp's parent (structure 669 from the structure_id_path). The structure tree can be used to generate a map from ids to names:
</div>

In [ ]:
name_map = structure_tree.get_name_map()
parent_id = structure_tree.parent_id([385])[0] # structure tree methods accept and return lists
name_map[parent_id]

In [ ]:
# The structure tree also has a built-in map for going from acronyms to ids:
acronym_to_id_map = structure_tree.get_id_acronym_map()
acronym_to_id_map['VISp']

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: mapping acronyms to names</h4>

<p>Write a function with the following signature:</p>
</div>

```python
def get_acronym_name_map(structure_tree):
    '''Builds a dictionary mapping structure acronyms to names.
    
    Parameters
    ----------
    structure_tree : StructureTree
        The complete structure tree.
        
    Returns
    -------
    acronym_name_map : dict
        Keys are acronyms (str); values are names (str).
    
    '''
    
    # your code here
    
    return acronym_name_map
```

In [ ]:
acronym_name_map = get_acronym_name_map(structure_tree)
acronym_name_map['VISp']

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: where is VISp in the structure tree?</h4>

Where is the primary visual area? One way to answer this question would be to name structures containing VISp. These are:
<ol>
<li>Visual Areas</li>
<li>Isocortex</li>
<li>...</li>
<li>root</li>
</ol>

In this excercise we will write a function that determines the nth ancestor of a given structure. Use the following signature:
</div>

```python
def where_in_the_tree(structure_tree, structure_name, steps=1):
    '''Names a structure which contains the supplied structure
    
    Parameters
    ----------
    structure_tree : StructureTree
        The tree in which structures will be looked up.
    structure_name : str
        The name of the structure whose ancestor we wish to find
    steps : int, optional
        How far rootward to walk. 1 gives the direct parent.
        
    Returns
    -------
    ancestor_name : str
        The full name of the requested ancestor structure.
    
    Examples
    --------
    >>>where_in_the_tree(structure_tree, 'Primary visual area', 2)
    u'Isocortex'
    >>>where_in_the_tree(structure_tree, 'Hippocampal formation', 0)
    u'Hippocampal formation'
    >>>where_in_the_tree(structure_tree, 'Cerebrum', 10000)
    None
    '''
    
    # your code here
    
    return ancestor_name

```

In [ ]:
print(where_in_the_tree(structure_tree, 'Primary visual area', 2))
print(where_in_the_tree(structure_tree, 'Hippocampal formation', 0))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h3><center>Working in 3d space</center></h3>

Anatomic variability across mice is a fundamental problem for our connectivity dataset. How can we ask questions about VISp if its location, size, and shape are different on every brain?

<p>We resolved this issue by generating an average template volume from 1675 mouse brain image stacks and registering each of our connectivity datasets to this average template. This allows us to work in a shared 3d space, which we call the Common Coordinate Framework</p>

<p>You can view the average template volume using your MouseConnectivityCache:</p>

</div>

In [ ]:
# load up the average template
template, header = mcc.get_template_volume()

# recall that this cache is at 25 micron isometric resolution
plt.imshow(template[200, :, :], interpolation='none', cmap=plt.cm.gray)

# how big?
print(template.shape)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>The template is oriented:</p>
<ul>
<li>anterior -> posterior</li>
<li>superior -> inferior</li>
<li>left -> right</li>
</ul>

<p>This means that to obtain a sagittal view, we must slice along axis 2:</p>
</div>

In [ ]:
plt.imshow(template[:, :, 125].T, interpolation='none', cmap=plt.cm.gray)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
and along axis 1 for a horizontal section:</div>

In [ ]:
plt.imshow(template[:, 75, :], interpolation='none', cmap=plt.cm.gray)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h4>The annotation volume</h4>

<p>Each (brain) voxel in the CCF is associated with a structure in the structure tree. This mapping is recorded in the annotation volume, which occupies the same space as the average template. The value of each voxel in the annotation volume is the id of the most specific structure occupying that point in space.</p>
</div>

In [ ]:
# you can obtain a reference space object from your mouse connectivity cache
# this object handles tasks that require both a structure tree and an annotation volume
reference_space =  mcc.get_reference_space()

# One such task is to apply our predefined structure colors to 
# a slice from the annotation volume
slice_image = reference_space.get_slice_image(1, 1875) # this method wants an axis and a position in microns
plt.imshow(slice_image, interpolation='none')

In [ ]:
# you can load up the raw annotation using your MouseConnectivityCache
annotation, _ = mcc.get_annotation_volume()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We want to find VISp, whose structure id is 385. How about:
</div>

In [ ]:
np.where(annotation == 385)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
Why did nothing show up? Recall that the annotation only labels the finest structure division at each voxel. VISp has several substructures:
</div>

In [ ]:
visp_children = structure_tree.children([385])[0]
pd.DataFrame(visp_children)

In [ ]:
# VISp, like other cortical structures, is subdivided into layers. 
# Voxels are directly assigned to the layer structures instead of to VISp
np.where(annotation==821) # 821 <==> VISp layer 2/3

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
A couple of caveats regarding the annotation:
<ol>
<li>It is possible for a structure and its substructures to all be directly annotated. An example would be hypothalamus, which has nuclei labeled where and only where they are distinctly identifiable.</li>
<li>Not all structures in the structure tree are annotated or have annotated substructures.</li>
</ol>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h4>Structure masks</h4>

<p>The annotation volume and structure tree contain a lot of information in a compact form. It can be simpler to work with more explicit representations of individual structures. We often use binary indicator masks for specific structures. These are:</p>
<ul>
<li>volumes in the common coordinate framework</li>
<li>1 where the annotation is labeled with a structure or its descendants, otherwise 0</li>
</ul>
</div>

In [ ]:
# view a structure mask for VISp
visp_mask, _ = mcc.get_structure_mask(385)
plt.imshow(visp_mask[:, :, 125].T, interpolation='none', cmap=plt.cm.gray)
plt.imshow(template[:, :, 125].T, interpolation='none', cmap=plt.cm.gray, alpha=0.4)

In [ ]:
# we can now easily locate VISp-labeled voxels in space:
np.where(visp_mask)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
It is easy to mask out data. The following syntax will do:
</div>

In [ ]:
visp_template = template.copy()

# we use 0 here for the masked value because pyplot.imshow doesn't play nice with nan.
# This is not necessarily the right choice for your application!
visp_template[visp_mask == 0] = 0
plt.imshow(visp_template[:, :, 125].T, interpolation='none', cmap=plt.cm.gray)

In [ ]:
# We can now ask questions about the template while restricting our inquiry to VISp
# for instance, to find the mean voxelwise template intensity within visp:
visp_template.sum() / np.count_nonzero(visp_mask)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: Regional template statistics</h4>

Write a function that computes percentiles of average template intensities in selected structures. Use it to compare the median intensity in white and gray matter.


hint: check out <a href="https://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.percentile.html">numpy's percentile function</a>
</div>

```python
def get_template_intensity_percentiles(mouse_connectivity_cache, structure_ids, n=None):
    '''Computes the nth percentile(s) of the average template volume's intensity within a particular subset of structures.
    
    Parameters
    ----------
    mouse_connectivity_cache : MouseConnectivityCache
    structure_ids : array-like of int
    n : array-like of float, optional
    
    Returns
    -------
    percentiles : array-like of float
        The nth percentiles. Uses the lower value when a percentile falls between two data.
    
    '''
    
    # your code here
    
    return percentiles
    
```

In [ ]:
# find the appropriate ids
structure_tree.get_structures_by_name(['fiber tracts', 'Basic cell groups and regions'])

In [ ]:
gm = get_median_template_intensity(mcc, [8])
wm = get_median_template_intensity(mcc, [1009])

print('gray: ' + str(gm))
print('white: ' + str(wm))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<center><h3>Which experiments have injections into VISp?</h3></center>

<p>Now that we have located primary visual cortex in the ontology tree and in physical space, we are ready to look at some connectivity experiments.</p>
<p>On <a href="http://connectivity.brain-map.org">connectivity.brain-map.org</a> you can do a <i>source search</i> to list experiments by the region into which viral tracer was injected. It is possible to do a similar search programmatically using the SDK.</p>
</div>

In [ ]:
# get a table of VISp experiments. This is a list of dictionaries.
visp_experiments = mcc.get_experiments(injection_structure_ids=[385])

# get a count
print(len(visp_experiments))

In [ ]:
# look at one of them
visp_experiments[150]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
Some particularly important fields here:
<ul>
<li>id : Like structures, each experiment has a unique id.</li>
<li>injection-structures : This is a list of structures that overlapped the injection site. VISp comes first, indicating that it overlapped most heavily. We call VISp the "primary injection site" for this experiment.</li>
<li>transgenic-line : Is this a wildtype mouse, or is fluorescence genetically restricted to a subset of neurons? This mouse is of the Scnn1a-Tg3 line. If it were wildtype, this field would be an empty string.</li>
</ul>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #f0f0ff; ">
<h4> Website/SDK Exercise: Locate an experiment on the website using the experiment id</h4>

<p>Find the experiment above on <a href="http://connectivity.brain-map.org">connectivity.brain-map.org</a> and verify that the primary injection structures and transgnic line information matches what you see here.</p>
</div>

<i>The easiest way to do this is to click on any experiment and open its "info" tab by clicking on the i in the top right corner. In the new window that opens up after clicking, paste your experiment id in place of the existing experiment id</i>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #f0f0ff; "><h4> Website/SDK Exercise: Find the number of VISp injection experiments that have been performed.</h4>
<p>Verify that you get the same answer using a website search and using the SDK
</div>

<i>This question can be answered either by doing a source search on http://connectivity.brain-map.org or using len(visp_experiments)</i>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: Transgenic line counts</h4>

<p>Many of our projection experiments are carried out in transgenic mice. Infected neurons in these mice fluoresce only if they also express a specific gene. Determine how many VISp experiments were carried out in each transgenic line.</p>

<p>note: we use an empty string for wildtype</p>
</div>

In [ ]:
line_counts

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #f0f0ff; "><h4> Website/SDK Exercise: Find the number of VISp injection experiments that have been performed.</h4>
<p>Cross-check <b>one</b> of these transgenic line experiment counts with what you find on the Allen Institute website.</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<center><h3>Gridded projection data</h3></center>

<p>Since each connectivity experiment is registered to the CCF, we can load connectivity data in the same way as we do the average template or annotation volume.</p>

Here we will load an experiment's projection density volume. These data:
<ul>
<li>are floats in [0, 1]</li>
<li>represent the fraction of each voxel that is occupied by signal (as detected in the segmentation stage)</li>
<li>are independent of the injection site (so it makes sense to discuss "projection in the injection site")</li>
</ul>
</div>

In [ ]:
prdens, _ = mcc.get_projection_density(159550125)

In [ ]:
def axis_proj(data, template, axis, bg_q=90, bg_alpha=0.5):
    # let's make some pictures
    
    data_max = np.amax(data, axis=axis)
    template_bg = np.percentile(template, axis=axis, q=bg_q)
    
    plt.imshow(data_max, interpolation='none', cmap=plt.cm.afmhot)
    plt.imshow(template_bg, interpolation='none', cmap=plt.cm.gray, alpha=0.5)

In [ ]:
# make a max projection along the anterior-posterior axis for visualization
axis_proj(prdens, template, 0)

In [ ]:
# now filter the signal to thalamus
thalamus_mask, _ = mcc.get_structure_mask(549)

prdens_th = prdens.copy()
prdens_th[thalamus_mask == 0] = 0
axis_proj(prdens_th, template, 0)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Another useful data volume is the injection fraction. This shows the fraction of each voxel that lies within this experiment's manually annotated injection site
</div>

In [ ]:
ijfr, _ = mcc.get_injection_fraction(159550125)

In [ ]:
axis_proj(ijfr, template, 0)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: Structure-level density</h4>

Write a function that determines the density of signal within a structure (i.e. the fraction of pixels in all the voxels in that structure that were detected as signal). 

In this experiment, what is the density in VISp? What about thalamus?
</div>

```python
def fraction_of_structure(mouse_connectivity_cache, structure_id, experiment_id):
    '''Computes the fraction of pixels in a specified structure that were detected as signal.

    Parameters
    ----------
    mouse_connectivity_cache : MouseConnectivityCache
    structure_id : int
    experiment_id : int

    Returns
    -------
    fraction : float

    '''
    
    # your code here

    return fraction

```

In [ ]:
print('thalamus', fraction_of_structure(mcc, 549, 159550125))
print('VISp', fraction_of_structure(mcc, 385, 159550125))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<center><h3>Which structures recieve the most projections from VISp?</h3></center>

<p>In order to answer this question you will need a set of structurewise projection summaries. One way to obtain these summary values would be to apply functions like the one you made in the previous excercise across a wider range of structures and experiments. This would be quite time-consuming, so we have precomputed such values for all experiments and structures.</p>

<p>We call these structurewise summary records "unionizes." Each unionize summarizes projection data over a unique (though potentially overlapping) spatial domain. Here are some unionize records:</p>
</div>

In [ ]:
sample_experiment = 159550125

unionizes = mcc.get_structure_unionizes(experiment_ids=[sample_experiment])
unionizes.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
Each row in this table is a different unionize record. Some of the columns tell you what spatial domain the unionize record summarizes over:
<ul>
<li>structure_id : Which structure does the unionize summarize over?</li>
<li>hemisphere_id* : Does the unionize summarize over the left hemisphere, the right, or both?</li>
<li>is_injection : Does the unionize correspond the the part of this structure which is within the injection site?</li>
</ul>
<p>The "experiment_id" column identifies the experiment from which this unionize's data was drawn. This is not to be confused (!) with the "id" column, which is a unique identifier for this specific unionize record.</p>

<p>All of the other fields are summaries of data. You can find a reference for these <a href='http://alleninstitute.github.io/AllenSDK/unionizes.html'>here</a>. </p>

<p>&#42; the hemisphere ids are: 1 => left, 2 => right, 3 => both</p>
</div>

In [ ]:
# You can also obtain al of the unionizes for a given set of experiments
wt_visp_experiment_ids = [exp['id'] for exp in visp_experiments if exp['transgenic-line'] == '']
wt_visp_unionizes = mcc.get_structure_unionizes(wt_visp_experiment_ids)
print(wt_visp_unionizes.shape)

In [ ]:
# numeric structure ids are hard to read. Let's use our structure name map from before to 
# add a structure name column to this unionize table
wt_visp_unionizes['structure_name'] = wt_visp_unionizes['structure_id'].map(lambda x: name_map[x])

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: Filtering to injection unionizes</h4>

<p>Create a new table which has the same unionize records as wt_vis_unionizes, but filtered down to only those where is_injection is True.</p>

<p>Hint: see the help for MouseConnectivityCache.get_structure_unionizes</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h4>A nonoverlapping set of structures</h4>

<p>A question like "which structures recieve the most projections from VISp" don't make much sense across the whole tree - some structures contain others and will therefore end up higher on the list.</p>

<p>We've created a nonoverlapping list of 293 biologically relevant structures, which we call the "summary structures." You can download a list of summary structures using the ontologies api:</p>
</div>

```python
oapi = OntologiesApi()
summary_structures = oapi.get_structures(structure_set_names="'Mouse Connectivity - Summary'")
summary_structure_ids = [item['id'] for item in summary_structures]
```

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
Because of our bandwidth limitations, we will find the summary structure ids using a function from the MouseConnectivityCache structure_tree method. Using the get_structures_by_set_id function, we can access any of the Allen Institute's predefined structure sets by their unique id, which can be found <a href='http://api.brain-map.org/api/v2/data/StructureSet/query.json?'>here</a>. For the "Mouse Connectivity - Summary" structure set, the unique id is 167587189.
</div>

In [ ]:
summary_structures = structure_tree.get_structures_by_set_id([167587189])
summary_structure_ids = [item['id'] for item in summary_structures]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Now that we have this set of structure ids, we can easily filter our unionizes to just those whose structure is a summary structure.

</div>

In [ ]:
wt_visp_summary_unionizes = (
    wt_visp_unionizes[wt_visp_unionizes['structure_id'].isin(summary_structure_ids)])
print(wt_visp_summary_unionizes.shape)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h4>What are the projection targets of VISp?</h4>

<p>Now that we have a dataframe containing the unionize data outside the injection site for all our experiments in wild type mice, we can use some pandas functions to look at our results. Pandas makes it easy to:</p>
<ol>
<li>Find the mean projection strength by structure across experiments using <a href='https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html'>pandas.DataFrame.groupby</a> combined with <a href='https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.mean.html'>pandas.DataFrame.mean</a></li>
<li>Find the regions that receive the strongest projections using <a href='https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html'>pandas.DataFrame.sort_values</a></li>
<li>Graph the data</li>
</ol>
</div>

In [ ]:
def top_projection(unionizes, measure, n, group_by='structure_name'):
    '''A helper function for listing the top n structures by the average of some data measure
    '''
    
    grouped = unionizes.groupby(group_by, as_index=False)[measure].mean()
    return grouped.sort_values(by=measure, ascending=False)[:n]

In [ ]:
# Since we are looking at projection targets, we want is_injection to be False
# When we have multiple conditions on a pandas dataframe, they can be chained together in a single slice

wt_visp_summary_projection_unionizes = (
    wt_visp_unionizes[(wt_visp_unionizes['is_injection'] == False) & 
                      (wt_visp_unionizes['structure_id'].isin(summary_structure_ids))
                     ])

In [ ]:
# get the top 20 targets of VISp using our helper function
top20_density = top_projection(wt_visp_summary_projection_unionizes, 'projection_density', 20)
top20_density['structure_name']

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">This list is a little hard to read. Let's make a bar plot.</div>

In [ ]:
def plot_mean_values(un, structure_names, measures, colors=None, include_stdev=False, plot_height=4):
    '''A helper function for making shared-axis plots from structure unionize data
    
    un : pd.DataFrame
        A table of unionizes. It must have a 'structure_name' column.
    structure_names : list of str
        include only data from these structures
    measures : list of str
        Each specifies a kind of data (unionize column). One plot will be generated per measure.
    colors : list of str, optional
        The color of the bars in each plot. If None (default), all of them will be dark blue.
    include_stdev : bool
        If true (default false), standard-deviation based error bars will be added to the plots.
    plot_height : numeric, optiona
        Height of each subplot in inches (your experience may vary). Default is 4.
    '''

    nplots = len(measures)
    
    if colors is None:
        colors = ['darkblue'] * nplots
    
    structure_names = list(structure_names)
    un = un[un['structure_name'].isin(structure_names)]
    
    fig, ax = plt.subplots(nplots, 1, sharex=True, figsize = (15, plot_height * len(measures)))
    for ii, mm in enumerate(measures):
        grouped = un.groupby('structure_name', as_index=False)
        gmeans = grouped[mm].mean()
        
        if nplots == 1:
            axis = ax
        else:
            axis = ax[ii]
        
        if include_stdev:
            gdevs = grouped[mm].apply(np.std, ddof = 0)
            gmeans.plot.bar('structure_name', mm, ax=axis, color=colors[ii], yerr=[gdevs])
        else:
            gmeans.plot.bar('structure_name', mm, ax=axis, color=colors[ii])
            
        axis.set_ylabel(mm)

In [ ]:
names_of_interest = set(top20_density['structure_name'])
plot_mean_values(wt_visp_summary_projection_unionizes, names_of_interest, 
                 ['projection_density'], ['teal'])

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4> Excercise: Comparing projection density and projection volume</h4>

<p>Use the above function to find the top 20 targets of VISp by normalized projection volume.</p>
<ul>
<li>Are these regions different from the top 20 targets by density?</li>
<li>Which targets are different? Hint: try casting the name column to a python set and using the "-" operator.</li>
<li>Which gives us the correct answer for the top 20 projection targets of primary visual cortex?</li>
</ul>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">

Now plot both sets of results (you can use the above plotting function). Use all structures that appeared in either of the top-20 lists.

<p>Recall that | is the union operator on python sets.</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; "><h4>Why do we get different answers with projection density and normalized projection volume?</h4>
<ul>
<li>projection_volume is the volume of projecting signal in mm3 (detected pixels)</li>
<li><u>normalized_projection_volume</u> is the projection_volume / volume of injection site</li>
<li><u>projection_density</u> is the sum of all detected pixels / volume of structure</li>
<li>projection_energy: sum of detected pixel intensity / volume of structure (in general, do not use this)</li>
</ul>

<p>Please see <a href='http://help.brain-map.org/display/mouseconnectivity/API#API-ProjectionDataGridding'>the documentation</a> for more information.</p>

<p>When deciding which measurement to use, think about whether your question is source-centric or target-centric and about how structure sizes might affect your results </p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: Summarizing across experiments</h4>

Write a function that computes a summary statistic across all experiments within a particular spatial domain. Pick a non-VISp structure and evaluate the normalized projection volume (volume of projection signal / volume of injection signal) in that structure across all VISp experiments in wild type mice.

hint: passing a function as an argument seems weird, but you can actually just call the resulting object. For instance, in the below signature calling fn(x) is equivalent to calling np.mean(x)
</div>

```python
def signal_summary(unionizes, structure_id, hemisphere_id, is_injection, signal_key, fn=np.mean):
    '''Applies a function to all unionizes describing a specified spatial domain.
    
    Parameters
    ----------
    unionizes : pd.DataFrame
    structure_id : int
        filter unionizes by structure
    hemisphere_id : int
        filter unionizes by hemisphere
    is_injection : bool
        filter unionizes by injection status
    signal_key : str
        select this column from the data
    fn : function, optional
        apply this function to the values of the resulting pandas series. Defaults to np.mean.
    
    Returns
    -------
    summary : numeric
        output of fn on filtered unionizes
    
    '''
 
    # code here
    
    return summary

```

In [ ]:
signal_summary(wt_visp_unionizes, 170, 3, False, 'normalized_projection_volume')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise: projection pattern comparisons</h4>

<p>Use the grid and/or unionize data to run a comparison of projection patterns between experiments varying along axes of your choice. Some options:</p>
<ul>
<li>cre line</li>
<li>injection hemisphere</li>
<li>primary injection structure</li>
<li>secondary injection structures, given a common primary injection structure</li>
</ul>
<p>As outputs you might look at:</p>
<ul>
<li>which structures are projected to</li>
<li>are projections equally strong across hemispheres?</li>
<li>do projections preferentially end up in subregions of the target structures? Can these subregions be identified as substructures in the tree, or only geographically?</li>
<li>Which regions are the most or least variably projected to by projections from a particular source?</li>
</ul>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h3><center>Additional excercises</center></h3>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; ">
<h4> Excercise 1: most recent common ancestor</h4>

<p>The goal of this excercise is to write a function with the following signature:</p>
</div>

```python
def get_most_recent_common_ancestor(structure_tree, structure_ids):
    '''Finds the most leafward structure that is an ancestor (inclusive) of a set of supplied structures.
    
    Parameters
    ----------
    structure_tree : StructureTree
        The complete structure tree.
    structure_ids : list of int
        One or more structure identifiers 
    
    Returns
    -------
    common_ancestor_id : int
        Structure id of the most specific ancestor of all supplied structures.
    
    Examples
    --------
    >>> get_most_recent_common_ancestor(structure_tree, [385, 425])
    669
    >>> get_most_recent_common_ancestor(structure_tree, [315])
    315
    >>> get_most_recent_common_ancestor(structure_tree, [])
    None
    
    '''

    # your code here

    return common_ancestor_id
```
<p><i>Hint: see StructureTree.ancestor_ids. How are the ids returned by this function ordered?</i></p>

In [ ]:
print(get_most_recent_common_ancestor(structure_tree, [385, 425]))
print(get_most_recent_common_ancestor(structure_tree, [315]))
print(get_most_recent_common_ancestor(structure_tree, []))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4> Excercise 2: Variability between experiments</h4>

<p>Find the standard deviation and/or SEM for the projection densitiy and normalized projection volume in the top 20 sources plotted above. Add error bars to the plot.</p>

<p>Which of these measures is more reliable? What about if you also check the (non-normalized) projection_volume?</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4> Excercise 3: Correlation between experiments</h4>
<p>Get the projection density volumes for two experiments and compute the correlation coefficient between the two.</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4>Exercise 4: Comparing wild type projection data with cre-defined projection data.</h4>
<p>Pick a region that has injection experiments in both wild type and cre mice. Compare the projections from the cre injection to the wild type projections using the visualization tool of your choice (bar plot, box plot, axis projection, matrix, etc...).</p>
<ul>
<li>Bonus 1: How far apart in space are the cre and wild type injections? Would you consider them to be well-matched?</li>
<li>Bonus 2: Are there any cre lines which have more projection targets than their corresponding wild type experiments? If so, is this true only in some regions or is it a general phenomenon?</li>
</ul>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4>Exercise 5: Relationship between structure size and projection density</h4>

<p>Because projection density is normalized by the volume of target structures, smaller structures that receive fewer projections overall may show a greater disparity between normalized_projection_volume and projection_density. Is this true for the targets of VISp? What is the relationship between structure size and projection density?</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4>Exercise 6: Structure based correlation search</h4>
<ul>
<li>Pick several experiments delivered into MOp.</li>
<li>Get the projection density values for all structures at the "Summary" level</li>
<li>Build an experiment-experiment correlation matrix based on those values.</li>
</ul>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4>Exercise 7: Reciprocal connections</h4>

<p>Of the top 20 projection targets of VISp (or your favorite region), which ones show reciprocal connections (project back to VISp)? Is the strength of the feedback projections proportional to the strength of the feedforward projections?</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FFF0; "><h4> Excercise 8: Generating a Projection Matrix</h4>

<p>The Mouse Connectivity Cache has a built-in function for generating a projection matrix (example below). Use the functions in structure_tree to generate matrices at different levels of detail. Make a projection matrix for:</p>
<ul>
<li>Contralateral projections from VISp</li>
<li>The 293 summary structures</li>
<li>Structures in the thalamus</li>
<li>All descendants of VISp</li>
</ul>
</div>

In [ ]:
ctx_children = [struct['id'] for struct in structure_tree.children([acronym_to_id_map['Isocortex']])[0]]

pm = mcc.get_projection_matrix(experiment_ids = wt_visp_experiment_ids, 
                               projection_structure_ids = ctx_children,
                               hemisphere_ids= [2], # right hemisphere, ipsilateral
                               parameter = 'projection_density')

row_labels = pm['rows'] # these are just experiment ids
column_labels = [ c['label'] for c in pm['columns'] ] 
matrix = pm['matrix']

fig, ax = plt.subplots(figsize=(12,12))
heatmap = ax.pcolor(matrix, cmap=plt.cm.afmhot)

# put the major ticks at the middle of each cell
ax.set_xticks(np.arange(matrix.shape[1])+0.5, minor=False)
ax.set_yticks(np.arange(matrix.shape[0])+0.5, minor=False)

ax.set_xlim([0, matrix.shape[1]])
ax.set_ylim([0, matrix.shape[0]])          

# want a more natural, table-like display
ax.invert_yaxis()
ax.xaxis.tick_top()

ax.set_xticklabels(column_labels, minor=False)
ax.set_yticklabels(row_labels, minor=False)
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h3><center>Ideas for shareable functions</center></h3>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<ol>
<li>Find the top n targets for an input source region</li>
<li>Find the top n sources that project to an input target region (or sources with projections stronger than a specified threshold)</li>
<li>Find the top n differences between two or more experiments</li>
<li>Quantify projection density by layer from a specified source to one or more target regions</li>
<li>Find all experiments within a specified distance of an anchor experiment
</ol>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h3><center>Project Ideas</center></h3>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h4>Projection Patterns</h4>
<ul>
<li>What are the pathways through the brain from sensory input to motor output?</li>
<li>What are the paths through the brain for incoming sensory information of different modalities? Where do they converge?</li>
<li>Hierarchical streams of cortical processing (for example in the visual cortex) is a successful model in the primate (Felleman and Van Essen). Is there evidence for a similar organization in the mouse (in the connectivity data set)?</li>
</ul>

<h4>Cell Types</h4>
<ul>
<li>Which layers and cre lines have contralateral projections? Does the density of contralateral projections vary by region? How do the targeted contralateral regions differ from targeted ipsilateral regions?</li>
<li>What do different cell types contribute to the network topology?</li>
</ul>

<h4>Topology</h4>
<ul>
<li>Many brain structures have known cortical projection topography including thalamus, superior colliculus, striatum, and primary motor cortex. Does an informatically-driven structure atlas look different when using projection-as-source data to define regions, as opposed to projection-as target (i.e. is there a difference in topology for incoming and outgoing projections in some regions)?</li>
<li>What structural motifs are present in the connectivity data set? How do they differ across regions?</li>
<li>What are the functional correlates to the structural connectivity? How do they relate to models of information processing?</li>
<li>Starting at a given point in the brain, what are the loop back routes and motifs to traverse the network and return to the same spot?</li>
<li>If you lose one particular cell type or cre line (say, due to a particular neurodegenerative disease), what happens to the routes and motifs in the rest of the network?</li>
</ul>

<h4>Methodology</h4>
<ul>
<li>In some cases, replicate experiments have very similar projection patterns while in other cases the difference between experiments is quite high. There are many experimental factors that contribute to this variability including the precise locations of injections, differences in injection size and depth, etc. Are there ways to quantify and/or control for these experimental variables in order to get a sense for the biological variability between animals? Which factors produce the largest differences between experiments (and are therefore most important to control when comparing across experiments)?</li>
<li>Most of the viral tracing experiments in the Allen Mouse Brain Connectivity Atlas use cytosolic GFP as a fluorescent indicator. This tracer produces fluorescent signal in axonal fibers as well as in synapses, so the quantified values are contaminated to some extent by fibers of passage. We have performed ~100 tracing experiments using a synaptophysin-eGFP virus (filter for Syp-EGFP under "Tracer Type"), which is a chimeric protein that is trafficked to synapses and has lower fluorescence in axons. How does the quantified projection data from this synaptic tracer differ from the traditional GFP tracer? Are some regions more contaminated by fibers of passage than others? Remember that the sources of variability listed above still apply here.</li>
</ul>
</div>